In [1]:
%load_ext autoreload

from IPython.display import clear_output

In [2]:
%autoreload

import os
import sys
import pdb

import numpy as np
import cPickle

import tensorflow as tf

from utils import load_data
from model import StrSumModel
from run import run

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

# cli.py

In [4]:
flags = tf.app.flags

flags.DEFINE_string('visible_gpu', '4', 'visible_gpu')

dataname = 'sports_fined.pkl'
flags.DEFINE_string('datapath', os.path.join('data', dataname), 'datapath')
modeldir = 'model_rougel_fined'
modelname = 'sports'
flags.DEFINE_string('modeldir', modeldir, 'modeldir')
flags.DEFINE_string('modelpath', os.path.join(modeldir, 'sports'), 'modeldir')

flags.DEFINE_bool('discourserank', True, 'discourserank')
flags.DEFINE_float('damp', 0.9, 'damping factor of discourserank')

flags.DEFINE_integer('epochs', 1000, 'epochs')
flags.DEFINE_integer('log_period', 500, 'log_period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('norm', 1e-4, 'norm')
flags.DEFINE_float('grad_clip', 10.0, 'grad_clip')
flags.DEFINE_float('keep_prob', 0.95, 'keep_prob')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('dim_hidden', 256, 'dim_output')
flags.DEFINE_integer('dim_str', 128, 'dim_output')
flags.DEFINE_integer('dim_sent', 384, 'dim_sent')

flags.DEFINE_integer('beam_width', 10, 'beam_width')
flags.DEFINE_integer('batch_size', 8, 'batch_size')

flags.DEFINE_string('f', '', 'kernel')

In [5]:
config = flags.FLAGS
os.environ['CUDA_VISIBLE_DEVICES'] = config.visible_gpu

# main.py 

In [6]:
from rouge import rouge_n, rouge_l_sentence_level

In [7]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

In [8]:
num_examples,  train_batches, dev_batches, test_batches, embedding_matrix, vocab, word_to_id = load_data(config)

In [9]:
# config.n_embed, config.d_embed = embedding_matrix.shape
flags.DEFINE_integer('n_embed', embedding_matrix.shape[0], 'n_embed')
flags.DEFINE_integer('d_embed', embedding_matrix.shape[1], 'd_embed')
# config.maximum_iterations = max([max([d._max_sent_len(None) for d in batch]) for ct, batch in dev_batches])
maximum_iterations = max([max([d._max_sent_len(None) for d in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')
flags.DEFINE_integer('PAD_IDX', word_to_id[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_id[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_id[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_id[EOS], 'EOS_IDX')
# config.PAD_IDX, config.UNK_IDX, config.BOS_IDX, config.EOS_IDX = word_to_id[config.PAD], word_to_id[config.UNK], word_to_id[config.BOS], word_to_id[config.EOS]
# config.vocab = vocab

In [10]:
%autoreload

from model import StrSumModel

model = StrSumModel(config)
model.build()

In [11]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
gvi = tf.global_variables_initializer()
sess.run(gvi)
sess.run(model.embeddings.assign(embedding_matrix.astype(np.float32)))

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [-0.201 ,  0.3212, -0.027 , ...,  0.1667, -0.0982, -0.0186],
       ...,
       [-1.3045,  0.3266,  0.5752, ...,  0.5891, -0.035 , -0.6161],
       [ 0.0143,  0.353 , -0.5258, ...,  0.5506, -0.2468,  0.3779],
       [ 0.1307,  0.0051, -0.2055, ..., -0.11  , -0.2712,  0.3053]],
      dtype=float32)

In [12]:
def get_txt_from_idx(idxs, model, vocab):
    return [' '.join([vocab[idx] for idx in idxs if (idx != model.config.EOS_IDX and idx != model.config.PAD_IDX)])]

def get_txt_from_tokens(tokens):
    return [' '.join([token for token in l]) for l in tokens]

In [13]:
def get_rouge(o_tokens, r_tokens, mode):
    if o_tokens == '': return (0.0, 0.0, 0.0)
    rouge = rouge_l_sentence_level(o_tokens, r_tokens) if mode == 'l' else rouge_n(o_tokens, r_tokens, mode)
    return rouge[0]

def get_rouges(sess, model, batch, vocab, modes=[1, 2, 'l']):
    feed_dict = model.get_feed_dict(batch, mode='test')
    _output_token_idxs = sess.run(model.beam_output_token_idxs, feed_dict = feed_dict)
    rouges = []
    for r_d, o_d in zip(batch, _output_token_idxs):
        o_idxs = o_d[0] if len(o_d.shape) == 2 else o_d
        o_tokens = get_txt_from_idx(o_idxs, model, vocab)
        r_tokens = get_txt_from_tokens([r_d.summary_tokens])
        rouge_batch = tuple([get_rouge(o_tokens, r_tokens, mode) for mode in modes])
        rouges.append(rouge_batch)
    return rouges

In [14]:
def evaluate(sess, batches, model, vocab):
    losses, rouges = [], []
    for ct, batch in batches:
        feed_dict = model.get_feed_dict(batch, mode='test')
        loss_batch = sess.run(model.loss, feed_dict = feed_dict)
        rouge_batch = get_rouges(sess, model, batch, vocab)
        losses += [loss_batch]
        rouges += rouge_batch
        
    loss_mean = np.mean(losses)
    rouge_mean = tuple(np.mean(rouges, 0))
    return loss_mean, rouge_mean

In [ ]:
losses_train = []
loss_log = []
rouge_log = []
rouge_l_max = 0

saver = tf.train.Saver(max_to_keep=20)
if len(loss_log) == 0:
    import subprocess
    
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

for ct, batch in train_batches:
    feed_dict = model.get_feed_dict(batch)
    _, loss_train = sess.run([model.opt, model.loss], feed_dict = feed_dict)
    losses_train += [loss_train]
    if ct%config.log_period==0:
        loss_train = np.mean(losses_train)
        loss_dev, rouge_dev = evaluate(sess, dev_batches, model, vocab)
        rouge_l = rouge_dev[-1]

        if rouge_l > rouge_l_max:
            rouge_l_max = rouge_l
            loss_test, rouge_test = evaluate(sess, test_batches, model, vocab)
            saver.save(sess, config.modelpath, global_step=ct)

        loss_log += [(ct, loss_train, loss_dev, loss_test)]
        rouge_log += [rouge_dev + rouge_test]
        losses_train = []

        clear_output()
        for i in range(len(loss_log)): 
            print 'Step: %i | LOSS TRAIN: %.3f, DEV: %.3f, TEST: %.3f ' %  loss_log[i], 
            print '| DEV ROUGE-1: %.3f, -2: %.3f, -L: %.3f | TEST ROUGE: -1: %.3f, -2: %.3f, -L: %.3f' % rouge_log[i]
#         print_sample(sess, sample_batch, model)